<a href="https://colab.research.google.com/github/SWEN90006/tutorial-solutions/blob/main/SWEN90006_Tutorial_4_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SWEN90006 Tutorial 4 solutions

## Task 1
First, make sure you understand the program (expected to be done before the tutorial).

## Task 2

Determine the input domains and output domains for the function `bubble`.

Next, what are the input conditions, and what are the output conditions?


**Answer**:
The input domain for `bubble` can be obtained by looking at the
parameters of `bubble`: $int[] \times int \times int.$


The input conditions are also straight-forward: $data \in int[]$ and
$size \in int$. The third input is an integer $order \in \{0, 1\}$, 0 means descending, and 1 indicates ascending, but we
do want this to behave like $\leq$ or $<$. We already have this
for `up` and `down` and since we have assumed that we are limited to
passing an integer to `bubble`, then this is all we require.


## Task 3
Perform a static data-flow analysis on `bubble`. Draw a table such as in the lecture notes to complete this.

**Answer**:
A simple way to attack data-flow problems is to write out a table with
all the variables, and in which node of the control-flow graph each
variables is defined, referenced, and undefined. In practice, we would
create the entries in the table as we encountered each variable. We
assign one of the states (u, d, r) to each variable.


| Line                | data | size | order| count| pass | SIZE |
|------|-----------|------------|---------------|-------------|------------|------------|
| int SIZE = 10       |      |      |      |      |      | d    |
| int [] bubble(..)   | d    | d    | d    |      |      |      |
| int pass, count     |      |      |      |      |      |      |
| pass = 0            |      |      |      |      | d    |      | 
| pass < SIZE - 1 ?   |      |      |      |      | r    | r    | 
| count = 0           |      |      |      | d    |      |      | 
| count < SIZE - 1 ?  |      |      |      | r    |      | r    | 
| if (order == 0)     |      |      | r    |      |      |      | 
| if (up...)          | r    |      |      | r    |      |      |
| swap(...)           | rd   |      |      | r    |      |      | 
| if (down...)        | r    |      |      | r    |      |      | 
| swap(...)           | rd   |      |      | r    |      |      |  
| count++             |      |      |      | rd   |      |      | 
| pass++              |      |      |      |      | rd   |      | 
| return data         | ru   | u    | u    | u    | u    |      |


Now, let's look at the results. A quick scan down the column for
`data` shows that the sequence **drdr** etc for this variable, which does
not reveal any problems.

Next, we look at `size`. Looking down the column, we see that size is
defined, and then undefined, without ever being referenced. This is a
*d-u* anomaly. What is the problem? The problem lies in the fact that
we had intended to use `size` rather than `SIZE` in the body of the
function. However, the program compiles because `bubble` is declared
within the scope of `static final int SIZE = 10`.

Looking at other variables, we see both `pass` and `count` 
contain a **du** sequence from the end of `rd` down to the `u` at the last statement. Referring back to the code, we see that this is NOT a du-anomaly because `pass` is defined and then used in the comparator again.

The lesson from this is that we must be careful when using tables to identify anomalies, because part of the control-flow of the program is discarded. We have to either return to the algorithm, or add the control-flow into the table.

## Task 4
Roughly sketch a set of black box test cases using equivalence partitioning.

## Task 5
What extra information does your data-flow analysis give you that your
black-box test cases do not?

**Answer**:
The important part of this question is to note the *d-u* anomaly from
above. This anomaly cannot be revealed by testing the `main`+`bubble`
program, because the parameter ` size` is always passed the argument
`SIZE`, so the behaviour is equivalent. If the `bubble` program is
tested in isolation, it can be detected. The advantage of a static
data-flow analysis is that this anomaly will *always* be detected,
whereas using testing, it requires certain cases to be selected. If
these are not selected, it may be missed.